In [2]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [3]:
Baengnyeongdo = pd.read_csv('D:\\SNUlab\\Baengnyeongdo_crawling_1401_2112.csv', encoding = 'euc-kr', low_memory = False)
Baengnyeongdo

,date,PRES,HGHT,TEMP,DWPT,RELH,MIXR,DRCT,SKNT,THTA,THTE,THTV
0,47102 Baengnyeongdo Observations at 00Z 01 Ja...,993.0,158.0,4.6,-2.4,60.0,3.24,250.0,12.0,278.3,287.5,278.9
1,47102 Baengnyeongdo Observations at 00Z 01 Ja...,978.0,283.0,4.0,-4.0,56.0,2.92,255.0,13.0,278.9,287.3,279.4
2,47102 Baengnyeongdo Observations at 00Z 01 Ja...,925.0,738.0,-0.3,-4.5,73.0,2.97,275.0,17.0,279.0,287.5,279.5
3,47102 Baengnyeongdo Observations at 00Z 01 Ja...,901.0,948.0,-2.5,-5.1,82.0,2.92,280.0,19.0,278.8,287.2,279.3
4,47102 Baengnyeongdo Observations at 00Z 01 Ja...,872.0,1207.0,-4.1,-17.1,36.0,1.15,287.0,23.0,279.8,283.3,280.0
...,...,...,...,...,...,...,...,...,...,...,...,...
514376,47102 Baengnyeongdo Observations at 12Z 31 De...,61.0,19183.0,-58.7,-80.7,4.0,0.01,275.0,48.0,476.8,476.9,476.8
514377,47102 Baengnyeongdo Observations at 12Z 31 De...,58.0,19501.0,-57.7,-79.7,4.0,0.01,305.0,48.0,486.1,486.2,486.1
514378,47102 Baengnyeongdo Observations at 12Z 31 De...,54.3,19916.0,-56.3,-78.3,5.0,0.02,328.0,25.0,498.5,498.6,498.5
514379,47102 Baengnyeongdo Observations at 12Z 31 De...,54.0,NaN,NaN,NaN,NaN,NaN,330.0,23.0,NaN,NaN,NaN


In [4]:
b_date = Baengnyeongdo['date'].unique().tolist()
len(b_date)

5511

In [37]:
final = pd.DataFrame()

for date in tqdm(b_date): 
    tmp = pd.DataFrame(Baengnyeongdo.loc[Baengnyeongdo['date'] == date].reset_index(drop = True))

    tmp['height_start'] = tmp['HGHT']
    tmp['height_end'] = 0
    tmp['height_end2'] = 0
    tmp['height_end3'] = 0
    tmp['height_end4'] = 0

    tmp['thetav_start'] = tmp['THTV']
    tmp['thetav_end'] = 0
    tmp['thetav_end2'] = 0
    tmp['thetav_end3'] = 0
    tmp['thetav_end4'] = 0

    tmp['temp_start'] = tmp['TEMP']
    tmp['temp_end'] = 0
    tmp['temp_end2'] = 0
    tmp['temp_end3'] = 0
    tmp['temp_end4'] = 0

    tmp['inversion_300'] = 0
    tmp['inversion_500'] = 0
    tmp['inversion_700'] = 0

    tmp['inversion_300_2'] = 0
    tmp['inversion_500_2'] = 0
    tmp['inversion_700_2'] = 0

    tmp['inversion_300_3'] = 0
    tmp['inversion_500_3'] = 0
    tmp['inversion_700_3'] = 0

    tmp['inversion_300_4'] = 0
    tmp['inversion_500_4'] = 0
    tmp['inversion_700_4'] = 0

    tmp['inversion_300_combined'] = 0
    tmp['inversion_500_combined'] = 0
    tmp['inversion_700_combined'] = 0
    
    for i in range(5): 
        tmp['height_end'][i] = tmp['height_start'][i+1]
        tmp['height_end2'][i] = tmp['height_start'][i+2]
        tmp['height_end3'][i] = tmp['height_start'][i+3]
        tmp['height_end4'][i] = tmp['height_start'][i+4]

        tmp['thetav_end'][i] = tmp['thetav_start'][i+1]
        tmp['thetav_end2'][i] = tmp['thetav_start'][i+2]
        tmp['thetav_end3'][i] = tmp['thetav_start'][i+3]
        tmp['thetav_end4'][i] = tmp['thetav_start'][i+4]

        tmp['temp_end'][i] = tmp['temp_start'][i+1]
        tmp['temp_end2'][i] = tmp['temp_start'][i+2]
        tmp['temp_end3'][i] = tmp['temp_start'][i+3]
        tmp['temp_end4'][i] = tmp['temp_start'][i+4]

    tmp.loc[((tmp['thetav_end'] - tmp['thetav_start']) / (tmp['height_end'] - tmp['height_start']) > 0.005) & (tmp['height_end'] - tmp['height_start'] > 100) & (tmp['height_end'] < 400), 'inversion_300'] = 1
    tmp.loc[((tmp['thetav_end'] - tmp['thetav_start']) / (tmp['height_end'] - tmp['height_start']) > 0.005) & (tmp['height_end'] - tmp['height_start'] > 100) & (tmp['height_end'] < 600), 'inversion_500'] = 1
    tmp.loc[((tmp['thetav_end'] - tmp['thetav_start']) / (tmp['height_end'] - tmp['height_start']) > 0.005) & (tmp['height_end'] - tmp['height_start'] > 100) & (tmp['height_end'] < 800), 'inversion_700'] = 1

    tmp.loc[((tmp['thetav_end2'] - tmp['thetav_start']) / (tmp['height_end2'] - tmp['height_start']) > 0.005) & (tmp['height_end2'] - tmp['height_start'] > 100) & (tmp['height_end2'] < 400), 'inversion_300_2'] = 1
    tmp.loc[((tmp['thetav_end2'] - tmp['thetav_start']) / (tmp['height_end2'] - tmp['height_start']) > 0.005) & (tmp['height_end2'] - tmp['height_start'] > 100) & (tmp['height_end2'] < 600), 'inversion_500_2'] = 1
    tmp.loc[((tmp['thetav_end2'] - tmp['thetav_start']) / (tmp['height_end2'] - tmp['height_start']) > 0.005) & (tmp['height_end2'] - tmp['height_start'] > 100) & (tmp['height_end2'] < 800), 'inversion_700_2'] = 1

    tmp.loc[((tmp['thetav_end3'] - tmp['thetav_start']) / (tmp['height_end3'] - tmp['height_start']) > 0.005) & (tmp['height_end3'] - tmp['height_start'] > 100) & (tmp['height_end3'] < 400), 'inversion_300_3'] = 1
    tmp.loc[((tmp['thetav_end3'] - tmp['thetav_start']) / (tmp['height_end3'] - tmp['height_start']) > 0.005) & (tmp['height_end3'] - tmp['height_start'] > 100) & (tmp['height_end3'] < 600), 'inversion_500_3'] = 1
    tmp.loc[((tmp['thetav_end3'] - tmp['thetav_start']) / (tmp['height_end3'] - tmp['height_start']) > 0.005) & (tmp['height_end3'] - tmp['height_start'] > 100) & (tmp['height_end3'] < 800), 'inversion_700_3'] = 1

    tmp.loc[((tmp['thetav_end4'] - tmp['thetav_start']) / (tmp['height_end4'] - tmp['height_start']) > 0.005) & (tmp['height_end4'] - tmp['height_start'] > 100) & (tmp['height_end4'] < 400), 'inversion_300_4'] = 1
    tmp.loc[((tmp['thetav_end4'] - tmp['thetav_start']) / (tmp['height_end4'] - tmp['height_start']) > 0.005) & (tmp['height_end4'] - tmp['height_start'] > 100) & (tmp['height_end4'] < 600), 'inversion_500_4'] = 1
    tmp.loc[((tmp['thetav_end4'] - tmp['thetav_start']) / (tmp['height_end4'] - tmp['height_start']) > 0.005) & (tmp['height_end4'] - tmp['height_start'] > 100) & (tmp['height_end4'] < 800), 'inversion_700_4'] = 1

    tmp.loc[(tmp['inversion_300'] == 1) | (tmp['inversion_300_2'] == 1) | (tmp['inversion_300_3'] == 1) | (tmp['inversion_300_4'] == 1), 'inversion_300_combined'] = 1
    tmp.loc[(tmp['inversion_500'] == 1) | (tmp['inversion_500_2'] == 1) | (tmp['inversion_500_3'] == 1) | (tmp['inversion_500_4'] == 1), 'inversion_500_combined'] = 1
    tmp.loc[(tmp['inversion_700'] == 1) | (tmp['inversion_700_2'] == 1) | (tmp['inversion_700_3'] == 1) | (tmp['inversion_700_4'] == 1), 'inversion_700_combined'] = 1

    final = pd.concat([final, tmp], axis = 0)

  0%|          | 0/17176 [00:00<?, ?it/s]<ipython-input-37-d8114dcd61f4>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['height_end'][i] = tmp['height_start'][i+1]
<ipython-input-37-d8114dcd61f4>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['height_end2'][i] = tmp['height_start'][i+2]
<ipython-input-37-d8114dcd61f4>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['height_end3'][i] = tmp['height_start'][i+3]
<ipyth